In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


In [2]:
books= pd.read_csv("//kaggle/input/book-recommendation-dataset/Books.csv")
ratings = pd.read_csv("//kaggle/input/book-recommendation-dataset/Ratings.csv")
users = pd.read_csv("//kaggle/input/book-recommendation-dataset/Users.csv")


In [3]:
books.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
users.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
ratings.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271359,271360,271358,271360,271360,271357
unique,271360,242135,102023,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,13903,7535,2,2,2


In [7]:
ratings.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [8]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [9]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [10]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

lets get our data into required shape



In [11]:
ratings_name = ratings.merge(books ,on='ISBN')
ratings_name

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [12]:
no_of_rating = ratings_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
no_of_rating.rename(columns={'Book-Rating':'num_of_rating'},inplace=True)
no_of_rating

,Book-Title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [13]:
#lets find out average of ratings on each book

average_rating = ratings_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
average_rating.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
average_rating

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [14]:
popular_books = no_of_rating.merge(average_rating ,on='Book-Title')
popular_books

,Book-Title,num_of_rating,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [15]:
#lets see the book with most no of ratings
popular_books[popular_books['num_of_rating']>=500].sort_values('num_of_rating',ascending=False)


,Book-Title,num_of_rating,avg_rating
234951,Wild Animus,2502,1.019584
196326,The Lovely Bones: A Novel,1295,4.468726
183573,The Da Vinci Code,898,4.642539
5303,A Painted House,838,3.231504
199237,The Nanny Diaries: A Novel,828,3.530193
27922,Bridget Jones's Diary,815,3.527607
206502,The Secret Life of Bees,774,4.447028
52983,Divine Secrets of the Ya-Ya Sisterhood: A Novel,740,3.437838
204387,The Red Tent (Bestselling Backlist),723,4.334716
14392,Angels &amp; Demons,670,3.708955


# **1. Popularity based filtering**

In [16]:
#lets filter data based on average rating & most number of ratings tpo get the most piopular books
popular_books_df=popular_books[popular_books['num_of_rating']>=500].sort_values('avg_rating',ascending=False)
popular_books_df

,Book-Title,num_of_rating,avg_rating
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
80445,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652
219741,To Kill a Mockingbird,510,4.700000
183573,The Da Vinci Code,898,4.642539
196326,The Lovely Bones: A Novel,1295,4.468726
206502,The Secret Life of Bees,774,4.447028
204387,The Red Tent (Bestselling Backlist),723,4.334716
74750,Girl with a Pearl Earring,526,4.218631
233370,Where the Heart Is (Oprah's Book Club (Paperba...,585,4.105983
107962,Life of Pi,664,4.088855


From the above data frame we can see the most popular books
If we are vary our parameter no of rating we may get different books at top,
here i gave value no of ratings > 500

In [17]:
pop_books = popular_books_df.merge(books ,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Year-Of-Publication']]
pop_books.shape
pop_books

,Book-Title,Book-Author,Year-Of-Publication
0,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000
3,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999
5,To Kill a Mockingbird,Harper Lee,1988
13,The Da Vinci Code,Dan Brown,2003
19,The Lovely Bones: A Novel,Alice Sebold,2002
20,The Secret Life of Bees,Sue Monk Kidd,2003
26,The Red Tent (Bestselling Backlist),Anita Diamant,1998
27,Girl with a Pearl Earring,Tracy Chevalier,2001
28,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998
29,Life of Pi,Yann Martel,2002


These are the most popular books based on the ratings so this our results to this dataset on **Popularity Based Filtering**

#  **2. Lets build our recommender sysytem based on Colloborative Filtering**

While a building this recommemnder system consider these conditions

1.You will not consider every user we will only consider the user who has 200 above reviews on the book

2.We will consider only those books which has more than 50 ratings

In [18]:
ratings_name.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [19]:
a = ratings_name.groupby('User-ID').count()['Book-Title']>300
geninue_users = a[a].index
print(type(geninue_users))

geninue_users

# Here we are using boolean indexing to fetch the users who gave reviews for more than 200 books
# here i choose user id count > 300 ,this parameter could have less more for time being i choose 300

<class 'pandas.core.indexes.numeric.Int64Index'>


Int64Index([  2276,   3363,   4385,   6251,   6543,   6575,   7158,   7346,
              8681,   8936,
            ...
            270713, 271284, 273979, 274004, 274061, 274301, 274308, 275970,
            277427, 278418],
           dtype='int64', name='User-ID', length=498)

In [20]:
geniune_user =ratings_name[ratings_name['User-ID'].isin(geninue_users)]
geniune_user

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
15,77940,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
16,81977,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1030883,275970,1880837927,0,The Theology of the Hammer,Millard Fuller,1994,Smyth &amp; Helwys Publishing,http://images.amazon.com/images/P/1880837927.0...,http://images.amazon.com/images/P/1880837927.0...,http://images.amazon.com/images/P/1880837927.0...
1030884,275970,188717897X,0,The Ordeal of Integration: Progress and Resent...,Orlando Patterson,1998,Civitas Book Publisher,http://images.amazon.com/images/P/188717897X.0...,http://images.amazon.com/images/P/188717897X.0...,http://images.amazon.com/images/P/188717897X.0...
1030885,275970,1888889047,0,Pushcart's Complete Rotten Reviews &amp; Rejec...,Bill Henderson,1998,Pushcart Press,http://images.amazon.com/images/P/1888889047.0...,http://images.amazon.com/images/P/1888889047.0...,http://images.amazon.com/images/P/1888889047.0...
1030886,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...


In [21]:
b = geniune_user.groupby('Book-Title').count()['Book-Rating']>=50
filtered_rat = b[b].index
filtered_rat

Index(['1st to Die: A Novel', '2nd Chance', '4 Blondes', 'A Bend in the Road',
       'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Heartbreaking Work of Staggering Genius',
       'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))',
       'A Man in Full', 'A Map of the World',
       ...
       'Wicked: The Life and Times of the Wicked Witch of the West',
       'Wild Animus', 'Winter Moon', 'Winter Solstice', 'Wish You Well',
       'Without Remorse', 'Wuthering Heights',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=477)

In [22]:
final_df =geniune_user[geniune_user['Book-Title'].isin(filtered_rat)]
final_df


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
63,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
65,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
66,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
69,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
74,23768,0446520802,6,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
...,...,...,...,...,...,...,...,...,...,...
1025261,263877,0613335864,0,False Memory,Dean Koontz,2001,Sagebrush Bound,http://images.amazon.com/images/P/0613335864.0...,http://images.amazon.com/images/P/0613335864.0...,http://images.amazon.com/images/P/0613335864.0...
1025321,264082,0821750666,0,Paradise,Phoebe Conn,1995,Zebra Books,http://images.amazon.com/images/P/0821750666.0...,http://images.amazon.com/images/P/0821750666.0...,http://images.amazon.com/images/P/0821750666.0...
1026724,266865,0531001725,10,The Catcher in the Rye,Jerome David Salinger,1973,Scholastic Library Pub,http://images.amazon.com/images/P/0531001725.0...,http://images.amazon.com/images/P/0531001725.0...,http://images.amazon.com/images/P/0531001725.0...
1028777,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...


lets get our data to required shape in the pivot tabler or in the form matrix 

In [23]:
piv_tbl = final_df.pivot_table(index='Book-Title',columns='User-ID')
piv_tbl.fillna(0,inplace=True)
piv_tbl.head(500)

Book-Rating                \
User-ID                                                 2276   3363   4385     
Book-Title                                                                     
1st to Die: A Novel                                        0.0    0.0    0.0   
2nd Chance                                                10.0    0.0    0.0   
4 Blondes                                                  0.0    0.0    0.0   
A Bend in the Road                                         0.0    0.0    0.0   
A Case of Need                                             0.0    0.0    0.0   
...                                                        ...    ...    ...   
Without Remorse                                            0.0    0.0    0.0   
Wuthering Heights                                          0.0    0.0    0.0   
Zen and the Art of Motorcycle Maintenance: An I...         0.0    0.0    0.0   
Zoya                                                       0.0    0.0    0.0   
\O\" Is for Outlaw"                                        0.0    0.0    0.0   

                                                                         \
User-ID                                            6251   6543   6575     
Book-Title                                                                
1st to Die: A Novel                                   0.0    9.0    0.0   
2nd Chance                                            0.0    0.0    0.0   
4 Blondes                                             0.0    0.0    0.0   
A Bend in the Road                                    0.0    0.0    1.0   
A Case of Need                                        0.0    0.0    0.0   
...                                                   ...    ...    ...   
Without Remorse                                       0.0    0.0    0.0   
Wuthering Heights                                     0.0    0.0    0.0   
Zen and the Art of Motorcycle Maintenance: An I...    0.0    0.0    0.0   
Zoya                                                  0.0    0.0    0.0   
\O\" Is for Outlaw"                                   0.0    0.0    0.0   

                                                                         \
User-ID                                            7158   7346   8681     
Book-Title                                                                
1st to Die: A Novel                                   0.0    0.0    0.0   
2nd Chance                                            0.0    0.0    0.0   
4 Blondes                                             0.0    0.0    0.0   
A Bend in the Road                                    0.0    0.0    0.0   
A Case of Need                                        0.0    0.0    0.0   
...                                                   ...    ...    ...   
Without Remorse                                       0.0    0.0    0.0   
Wuthering Heights                                     0.0    0.0    0.0   
Zen and the Art of Motorcycle Maintenance: An I...    0.0    0.0    0.0   
Zoya                                                  0.0    0.0    0.0   
\O\" Is for Outlaw"                                   0.0    8.0    0.0   

                                                           ...                \
User-ID                                            8936    ... 270713 271284   
Book-Title                                                 ...                 
1st to Die: A Novel                                   0.0  ...    0.0    0.0   
2nd Chance                                            0.0  ...    0.0    0.0   
4 Blondes                                             0.0  ...    0.0    0.0   
A Bend in the Road                                    0.0  ...    0.0    0.0   
A Case of Need                                        0.0  ...    0.0    0.0   
...                                                   ...  ...    ...    ...   
Without Remorse                                       0.0  ...    0.0    0.0   
Wuthering Heights           

Here we have to caluclate eucledian distance for given data after that we can recommend book

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
cos_simscore = cosine_similarity(piv_tbl)
print(cos_simscore.shape)
cos_simscore


(477, 477)


array([[1.        , 0.268097  , 0.        , ..., 0.09253734, 0.20820078,
        0.1761926 ],
       [0.268097  , 1.        , 0.        , ..., 0.0592571 , 0.06411481,
        0.13948894],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.09253734, 0.0592571 , 0.        , ..., 1.        , 0.08532999,
        0.02351353],
       [0.20820078, 0.06411481, 0.        , ..., 0.08532999, 1.        ,
        0.12692859],
       [0.1761926 , 0.13948894, 0.        , ..., 0.02351353, 0.12692859,
        1.        ]])

lets us here create a function which will recommend us books

In [26]:
def recommend(books_name):
    index = np.where(piv_tbl.index==books_name)[0][0]    
    similar_books = sorted(list(enumerate(cos_simscore[index])),key=lambda x:x[1],reverse=True)[1:6]
    
    for i in similar_books:
        print(piv_tbl.index[i[0]])
    

In [27]:
recommend('The Da Vinci Code')

Angels &amp; Demons
Timeline
Misery
Saving Faith
Middlesex: A Novel


So lets our recommender system is it recommending books similarly

In [28]:
recommend('The Fellowship of the Ring (The Lord of the Rings, Part 1)')

The Two Towers (The Lord of the Rings, Part 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
The Return of the King (The Lord of the Rings, Part 3)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Goblet of Fire (Book 4)


From the above output we can say that our recommender system working quite well, i passed book 
The Fellowship of the Ring(The Lord of the Rings, Part 1) which is fantasy & fiction book ,

The recommender system suggested me the books The lord of the rings books sequel like 2&3 
and gave harry porter books sequels which are also fantasy fiction 

                                                                        The End 